In [ ]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
import multiprocessing as mp
import os
from build import build_model
import coralme
from tqdm import tqdm

In [ ]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

### Load

In [ ]:
def load_data(path):
    try:
        df = pandas.read_csv(path,index_col=0)[["fluxes"]]
    except:
        df = pandas.DataFrame(columns=["fluxes"])
    df.columns = [org]
    return df

In [ ]:
dataset = ""

In [ ]:
conditions = ["IBD","nonIBD"]
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
fluxes = {i:pandas.DataFrame() for i in conditions}
for org in tqdm(organisms):
    for c in conditions:
        tmp = load_data("./cases/fluxes/{}{}/{}.csv".format(c,dataset,org))
        tmp = tmp.loc[[i for i in tmp.index if i.startswith("EX_")]]
        fluxes[c] = pandas.concat([fluxes[c],tmp], axis=1).fillna(0)

In [ ]:
predictions = {
    "fluxes" : fluxes,
    "growth" : pandas.read_csv("./datasets/3.5.IBD{}_growth_rates.csv".format(dataset),index_col=0)
}

In [ ]:
RelativeAbundances = pandas.read_csv("./ibdmdb/Referenced_metaG_per_diagnosis.csv",index_col=0)

In [ ]:
FluxesByOrganism = {}
for c in predictions["growth"].columns:
    tmp = predictions["fluxes"][c].multiply(RelativeAbundances[c],axis=1)
    FluxesByOrganism[c] = tmp[tmp.any(axis=1)]

In [ ]:
for k,v in FluxesByOrganism.items():
    v.to_csv("./datasets/3.6.TotalFluxes_{}{}.csv".format(k,dataset))